## Functions trash bin

### Modifying file

In [38]:
import json
from os.path import exists

def get_data(file_path):
    if not exists(file_path):
        return {}
    
    with open(file_path, 'r') as f:
        return json.load(f)


def save_data(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

### Filtering data

In [64]:
def filter_data(**attributes):
    data = get_data()
    result = dict()

    for k, v in data.items():
        if not any(v[k1] not in v1 for k1, v1 in attributes.items()):
            result[k] = v
    
    return result

### Updating algs

In [60]:
import pandas as pd
import numpy as np
from pathlib import Path

VALID_CHARS = " UDFBRLMESudfbrlw'/:,2xyz"

def excel_to_dict_of_df(filename):
    my_dict = pd.read_excel(filename, header=None, sheet_name=None)
    my_dict = {k: df.replace(np.nan,'',regex=True) for k, df in my_dict.items()}
    return my_dict


def df_to_alg_list(df):
    buffer = df.iloc[0][0]
    if not buffer or df.shape[0] < 2 or df.shape[1] < 2 or not df.iloc[0][1]:
        return None

    piece_type = buffer_to_type(buffer)
    result = []

    # table
    if df.iloc[1][0] == df.iloc[0][1]:
        for i in range(1, df.shape[1]):
            for j in range(1, df.shape[0]):
                if df.iloc[i][j]:
                    alg = clean_alg_entry(df.iloc[i][j])
                    key = ";".join([df.iloc[0][0], df.iloc[0][j], df.iloc[i][0], alg])
                    result.append(key)
        return {piece_type: result}
    
    if not (df.shape[1] == 4 or df.shape[1] == 5):
        return None
    # list
    if df.shape[1] == 4:
        row = 1
        while df.iloc[row][0]:
            alg = clean_alg_entry(df.iloc[row][3])
            key = ";".join([df.iloc[row][0], df.iloc[row][1], df.iloc[row][2], alg])
            result.append(key)
            row += 1
        return {piece_type: result}
    
    # parity
    row = 1
    while df.iloc[row][0]:
        alg = clean_alg_entry(df.iloc[row][4])
        key = ";".join([df.iloc[row][0], df.iloc[row][1], df.iloc[row][2], df.iloc[row][3], alg])
        result.append(key)
        row += 1
    return {'parity': result}


def clean_alg_entry(alg):
    # filtering out invalid chars
    alg = ''.join([i for i in alg if i in VALID_CHARS])

    # cleaning multiple spaces 
    alg = ' '.join(alg.split())

    # cleaning whitespaces before , : '
    i = 0
    while i < len(alg):
        if alg[i] in "':," and i > 0 and alg[i - 1] == " ":
            alg = alg[:i-1] + alg[i:]
        else:
            i += 1

    return alg


def only_latest_algs(data):
    return {k: v for k, v in data.items() if v['latest_alg']}


def keys_with_different_algs(data, key):
    return [k for k in data if key.split(';')[:-1] == k.split(';')[:-1]]


def is_key_in_dict(data, key):
    return key in data


def new_record_from_key(key):
    key = key.split(';')
    record = {
        'buffer': key[0],
        'first_target': key[1],
        'second_target': key[2],
        'results': [],
        'latest': True
    }

    if len(key) == 4:
        record['alg'] = key[3]
        return record
    
    record['third_target'] = key[3]
    record['alg'] = key[4]
    return record
    


def update_algs(excel_path):
    sheets = excel_to_dict_of_df(excel_path)
    algs = dict()
    for k, df in sheets.items():
        algs_list = df_to_alg_list(df)

        if algs_list is None:
            continue
        
        print(k)

        for piece_type, alg_list in algs_list.items():
            algs[piece_type] = algs.get(piece_type, []) + alg_list
        
    for piece_type, algs_list in algs.items():
        filepath = Path().absolute().parent / 'json' / f'{piece_type}.json'
        data = get_data(str(filepath))
        
        for key in algs_list:
            existing_algs = keys_with_different_algs(data, key)

            for k in existing_algs:
                data[k]['latest'] = False

            if key in data:
                data[key]['latest'] = True
            else:
                data[key] = new_record_from_key(key)

        save_data(data, str(filepath))

def buffer_to_type(buffer):
    if len(buffer) < 2 or len(buffer) > 3:
        return 'error'
    
    if buffer[0].islower():
        return 'midges' + '_' + buffer.lower()
    
    if len(buffer) == 2 and buffer[1].islower():
        return 'tcenters' + '_' + buffer.lower()
    
    if len(buffer) == 2:
        return 'edges' + '_' + buffer.lower()
    
    if buffer[1].islower():
        return 'xcenters' + '_' + buffer.lower()
    
    if buffer[2].islower():
        return 'wings' + '_' + buffer.lower()
    
    return 'corners' + '_' + buffer.lower()

### Updating memo

In [91]:
def buffer_to_type(buffer, suffix = False):
        if len(buffer) < 2 or len(buffer) > 3:
            return "error"
        
        piece_type = ''

        if buffer[0].islower():
            piece_type = "midges"

        elif len(buffer) == 2 and buffer[1].islower():
            piece_type = "tcenters"

        elif len(buffer) == 2:
            piece_type = "edges"
        elif buffer[1].islower():
            piece_type = "xcenters"

        elif buffer[2].islower():
            piece_type = "wings"
        
        else:
            piece_type = "corners"

        if suffix:
            piece_type = f'{piece_type}_{buffer}'
        
        return piece_type
    

def df_to_words_dict(df):
    if df.shape[0] < 2 or df.shape[1] < 2 or not df.iloc[0][1]:
        return None

    piece_type = buffer_to_type(df.iloc[0][1])

    result = dict()
    if df.iloc[1][0] == df.iloc[0][1]:
        for i in range(1, df.shape[1]):
            for j in range(1, df.shape[0]):
                if df.iloc[i][j]:
                    key = f'{df.iloc[0][j]};{df.iloc[i][0]}'
                    result[key] = df.iloc[i][j]
        return {piece_type: result}


    row = 1
    while df.iloc[row][0]:
        key = f'{df.iloc[row][0]};{df.iloc[row][1]}'
        result[key] = df.iloc[row][2]
        row += 1
    return {piece_type: result}

import os
def update_words(filepath):
    words = excel_to_dict_of_df(filepath)
    words_dict = dict()
    for df in words.values():
        
        words_grouped_by_type = df_to_words_dict(df)
        
        if words_grouped_by_type is None:
            continue
        
        for piece_type, words_grouped_by_case in words_grouped_by_type.items():
            if piece_type in words_dict:
                words_dict[piece_type].update(words_grouped_by_case)
            else:
                words_dict[piece_type] = words_grouped_by_case
    

    for piece_type, words in words_dict.items():
        path_to_jsons = Path().absolute().parent / "json"

        jsons = []

        for filename in os.listdir(path_to_jsons):
            if filename.startswith(piece_type):
                jsons.append(filename)

        print(jsons)
        for filename in jsons:
            data = get_data(path_to_jsons / filename)

            for k, v in data.items():
                try:
                    targets = f"{v['first_target']};{v['second_target']}"
                    data[k]['word'] = words[targets]
                except KeyError:
                    pass
            
            save_data(data, path_to_jsons / filename)

In [92]:
update_words('memo.xlsx')

['edges_db.json', 'edges_df.json', 'edges_dl.json', 'edges_dr.json', 'edges_fl.json', 'edges_fr.json', 'edges_ub.json', 'edges_uf.json', 'edges_ul.json', 'edges_ur.json']


### Updating LPs

In [9]:
def get_lps_dict(grid):
    return {i[0]: i[1] for i in grid}

def update_LPs():
    lps = excel_to_dict_of_df('LP.csv')
    lps_dict = get_lps_dict(lps)

    for k, v in data.items():
        try:
            targets = [v['first_target'], v['second_target']]
            data[k]['lp'] = ''.join([lps_dict[i] for i in targets])
        except KeyError:
            pass
    
    return data

In [44]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json

def get_data(file_path):
    if not os.path.exists(file_path):
        return {}

    with open(file_path, "r") as f:
        return json.load(f)

def save_data(data, file_path):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)


def buffer_to_type(buffer, suffix = False):
        if len(buffer) < 2 or len(buffer) > 3:
            return "error"
        
        piece_type = ''

        if buffer[0].islower():
            piece_type = "midges"

        elif len(buffer) == 2 and buffer[1].islower():
            piece_type = "tcenters"

        elif len(buffer) == 2:
            piece_type = "edges"
        elif buffer[1].islower():
            piece_type = "xcenters"

        elif buffer[2].islower():
            piece_type = "wings"
        
        else:
            piece_type = "corners"

        if suffix:
            piece_type = f'{piece_type}_{buffer}'
        
        return piece_type

def excel_to_dict_of_dfs(filepath):
        my_dict = pd.read_excel(filepath, header=None, sheet_name=None)
        my_dict = {k: df.replace(np.nan, "", regex=True) for k, df in my_dict.items()}
        return my_dict

def df_to_lps_dict(df):
        if df.shape[0] < 2 or df.shape[1] < 2 or not df.iloc[0][1]:
            return None

        piece_type = buffer_to_type(df.iloc[1][0])
        result = dict()
        for i in range(df.shape[0]):
            result[df.iloc[i][0]] = df.iloc[i][1]
        return {piece_type: result}


def update_lps(filepath):
    lps = excel_to_dict_of_dfs(filepath)
    lps_dict = dict()
    for df in lps.values():
        
        lps_grouped_by_type = df_to_lps_dict(df)
        
        if lps_grouped_by_type is None:
            continue
        print(lps_grouped_by_type)
        for piece_type, words_grouped_by_case in lps_grouped_by_type.items():
            if piece_type in lps_dict:
                lps_dict[piece_type].update(words_grouped_by_case)
            else:
                lps_dict[piece_type] = words_grouped_by_case
    

    for piece_type, lps in lps_dict.items():
        path_to_jsons = Path().absolute().parent / "json"

        jsons = []

        for filename in os.listdir(path_to_jsons):
            if filename.startswith(piece_type):
                jsons.append(filename)

        for filename in jsons:
            data = get_data(path_to_jsons / filename)

            for k, v in data.items():
                try:
                    targets = [v['first_target'], v['second_target']]
                    data[k]['lp'] = ''.join([lps[i] for i in targets])
                except KeyError:
                    pass
            
            save_data(data, path_to_jsons / filename)

In [45]:
update_lps('lps.xlsx')

{'edges': {'UB': 'A', 'UL': 'B', 'UR': 'C'}}


### Running game

In [17]:
from random import shuffle
from datetime import datetime

def get_n_random_keys(data, n):
    data = {k: v for k, v in data.items() if v['latest']}
    keys = list(data)
    shuffle(keys)
    return keys[:n]    


def measure_time():
    start = datetime.now()
    input()
    end = datetime.now()
    delta = (end - start).total_seconds()
    return round(delta, 2)


def display_record(alg):
    if 'word' in alg:
        print(alg['word'])
    elif 'lp' in alg:
        print(alg['lp'])
    else:
        print(f"{alg['first_target']} {alg['second_target']}")


def get_response():
    print()
    response = input('Type Y to save results')
    return response == 'Y'


def add_entries(data, results):
    for k, v in results.items():
        data[k]['results'].append(v)

    save_data(data)

def run_game(n):
    results_dict = dict()
    data = get_data()
    algs = get_n_random_keys(data, n)
    for alg in algs:
        display_record(data[alg])
        exec_time = measure_time()
        results_dict[alg] = exec_time

    print('Results to be saved:')
    for k, v in results_dict.items():
        print(' '.join(k.split(';')[1:3]), v)

    if get_response():
        add_entries(data, results_dict)

### El śmietniko

In [66]:
query = {
    "latest": [False]
}

filtered_data = filter_data(**query)
for k, v in filtered_data.items():
    print(v['buffer'], v['first_target'], v['second_target'])
    print(v['alg'])
    print()

UF UR UB
R2 U: S, R2



In [43]:
key = "[Ra    :  R  (1E ' 6  R    '&&   , %9 U       ']"
clean_alg_entry(key)

"R: R E' R', U'"

In [68]:
save_data(update_words())

In [69]:
run_game(5)

Arbuz
Bat
Cable
Acid
Baca
Results to be saved:
UB UL 0.71
UL UB 0.92
UR UL 1.29
UB UR 2.81
UL UR 3.06



In [31]:
import pandas as pd

df = pd.read_excel('data.xlsx', sheet_name = None) # can also index sheet by name or fetch all sheets
df_uf = df['UF']

In [35]:
df_uf.iloc[0][0]

'UB'

In [37]:
df_uf.keys()[0]

'UF'

In [44]:
df2 = pd.read_csv('algs.csv', sep=';', header=None)

In [45]:
df2

,0,1,2,3
0,UF,UB,UL,UR
1,UB,NaN,"U' M2 U: U2, M",R2 U' S' U2 S U' R2
2,UL,"U' M2 U: M, U2",NaN,"M2 U: M, U2"
3,UR,"R2 U': S, R2","M2 U': M, U2",NaN


In [47]:
df2.iloc[2][1]

"U' M2 U: M, U2"

TypeError: 'float' object is not iterable

In [2]:
t1 = "R2 U': R2, S"
t1 = t1.split(':')
t1 = [i.strip().split(',') for i in t1]
t1

[["R2 U'"], ['R2', ' S']]

In [1]:
import re

class Move:
    def __init__(self, move):
        self.move = move
        self.set_base_move()
        self.set_suffix(''.join([i for i in self.move if not i.isalpha()]))
        self.set_axis()


    def get_move(self):
        return self.base_move + self.suffix

    def set_base_move(self):
        self.base_move = ''.join([i for i in self.move if i not in "'2"])

    def get_base_move(self):
        return self.base_move

    def set_suffix(self, suff):
        self.suffix = suff 

    def get_suffix(self):
        return self.suffix
    
    def get_inv_suffix(self):
        if self.suffix == "'":
            return ''
        if self.suffix == '':
            return "'"
        return self.suffix

    def set_axis(self):
        
        def axis_check(moves):
            return self.base_move[0] in moves
        
        if axis_check('UED'):
            self.axis = 'y'
        
        elif axis_check('LMR'):
            self.axis = 'x'

        else:
            self.axis = 'z'

    def get_axis(self):
        return self.axis

    def inverse_move(self):
        if self.suffix == "'":
            self.suffix = ''

        elif self.suffix == '':
            self.suffix = "'"
        
    def __repr__(self):
        return self.base_move + self.suffix
        

class ComutatorAnalyzer:
    def __init__(self, comm):
        self.comm = comm
        self.alg = self.expandComm()
    
    @staticmethod
    def list_to_alg(alg, inv=False):
        alg = [Move(i) for i in alg]

        if inv:
            alg = alg[::-1]
            for i in alg:
                i.inverse_move()

        return alg

    @staticmethod
    def splitted_comm_to_alg(a, b, c=''):
        base = ComutatorAnalyzer.list_to_alg(a) + ComutatorAnalyzer.list_to_alg(b) + ComutatorAnalyzer.list_to_alg(a, True) + ComutatorAnalyzer.list_to_alg(b, True)

        if c:
            return ComutatorAnalyzer.list_to_alg(c) + base + ComutatorAnalyzer.list_to_alg(c, True)
        
        return base

    def split_comm(self):
        split_by_sep = [i.strip() for i in re.split(':|,', self.comm)]
        return [i.split() for i in split_by_sep]

    @staticmethod
    def reduce_moves(m1, m2):
        if m1.get_axis() != m2.get_axis():
            return [m1, m2]
        
        def suff_to_pos(suffix):
            if suffix == '':
                return 1
            if suffix == '2':
                return 2
            return 3
        
        def suff_sum(s1, s2):
            pos = suff_to_pos(s1) + suff_to_pos(s2)
            return pos % 4

        def pos_to_move(base_move, pos):
            if pos == 0:
                return 
            if pos == 1: 
                return Move(base_move)
            if pos == 2:
                return Move(base_move + '2')
            return Move(base_move + "'")


        if m1.get_base_move() == m2.get_base_move():

            pos = suff_sum(m1.get_suffix(), m2.get_suffix())
            
            return [pos_to_move(m1.get_base_move(), pos)]
        
        if m1.get_axis() == 'x':
            order = ['R', 'Rw', 'M', 'Lw', 'L']

        elif m1.get_axis() == 'y':
            order = ['U', 'Uw', 'E', 'Dw', 'D']
        
        else:
            order = ['B', 'Bw', 'S', 'Fw', 'F']


        if order.index(m1.get_base_move()) > order.index(m2.get_base_move()):
            m1, m2 = m2, m1
        
        b1, b2 = m1.get_base_move(), m2.get_base_move()
        s1, s2 = m1.get_suffix(), m2.get_suffix()

        if b1 == order[0]:
            if s1 == m2.get_inv_suffix():
                if b2 == order[1]:
                    return [Move(order[2] + m2.get_inv_suffix())]
                if b2 == order[2]:
                    return [Move(order[1] + s1)]
            
            if b2 == order[1]:
                pos = suff_sum(s1, s2)
                return [pos_to_move(b1, pos), Move(order[2] + m2.get_inv_suffix())]

        if b1 == order[1] and b2 == order[2]:
            if s1 == s2:
                return [Move(order[0] + s1)]
            
            else:
                pos = suff_sum(s1, s2)
                return [Move(order[0] + s1), pos_to_move(b2, pos)]

        

        
        if b1 == order[2] and b2 == order[3] and s1 == m2.get_inv_suffix():
            return [Move(order[4] + m2.get_inv_suffix())]
        
        if b1 == order[2] and b2 == order[4] and s1 == s2:
            return [Move(order[3] + s2)]
        
        if b1 == order[3] and b2 == order[4] and s1 == m2.get_inv_suffix():
            return [Move(order[2] + s1)]
        
        return [m1, m2]
 

    def expandComm(self):
        splitted_comm = self.split_comm()

        a, b = splitted_comm[-2], splitted_comm[-1]
        c = splitted_comm[0] if splitted_comm[0] != splitted_comm[-2] else ''

        alg = ComutatorAnalyzer.splitted_comm_to_alg(a, b, c)

        i = 0
        while i < len(alg) - 1:
            reduced_pair = ComutatorAnalyzer.reduce_moves(alg[i], alg[i + 1])

            if reduced_pair != alg[i : i + 2]:
                alg = alg[:i] + reduced_pair + alg[i + 2:]
                print(alg)
                i = 0

            else:
                i+=1


        self.alg = alg

    def get_alg_str(self):
        return ' '.join([i.get_move() for i in self.alg])



        

In [2]:
def comm_to_alg(comm):
    ca = ComutatorAnalyzer(comm)
    ca.expandComm()
    return ca.get_alg_str()

In [ ]:
comms = [
    "Uw': L E' L', U2",
    "Uw': L U' L', E2",
    # "U' Rw': E, R U R'",
    # "U' M' U': M, U2",
    # "Uw': R' E R, U"
]

for comm in comms:
    print(f"{comm} -> {comm_to_alg(comm)}")
    

Uw': L E' L', U2 -> Uw' L E' L' U2 L E L' U' E'


In [51]:
a = Move('Rw')
b = Move('L2')
sorted([a,b], key=lambda x: x.get_move())

[L2, Rw]

In [20]:
comm = "R2 U': R2, S"
split_by_sep = [i.strip() for i in re.split(':|,', comm)]
[[Move(j) for j in i.split()] for i in split_by_sep]

[[R2, U'], [R2], [S]]

In [16]:
sorted(['U', 'Uw', 'E', 'D', 'Dw'], reverse=True)

['Uw', 'U', 'E', 'Dw', 'D']

In [4]:
to_find = 'd'
a = ['a', 'b', 'c', 'd', 'e', 'f']
i = a.index(to_find)
b = [a[i]] + a[:i] + a[i+1:]
b


['d', 'a', 'b', 'c', 'e', 'f']

In [3]:
a.index('c')

2

In [7]:
a[1:]

['b', 'c', 'd', 'e', 'f']